In [2]:
from visualization_fct import *
from sklearn.mixture import BayesianGaussianMixture

# from bokeh.plotting import output_file, show, save
# from bokeh.mpl import to_bokeh

from bokeh.resources import CDN
from bokeh.embed import file_html

import matplotlib.pyplot as plt  # , mpld3
import pyfits


# for now, no ratio in data (no rates A, B or C in this dataset)

only_flux = False
scale_data = True

hdulist = pyfits.open('../iirc_data/all_data_for_ml.fits')
data = hdulist[1].data

#data=data[500:]

X_flux, X, data_thr, data_fr_en = get_iirc_data(
    data, only_flux=only_flux, scale_data=scale_data, thresholded=True)[:4]

# VBGMM with 3 components: 
#np.random.seed(0)

XX = X[78:]

gmm = BayesianGaussianMixture(n_components=50, alpha_init=0.0000001, random_state=0, n_init=10).fit(XX)
preds = gmm.predict(XX)
probs = gmm.predict_proba(XX)

data_thr['preds'] = pd.Series(preds).astype("category")

color_key = ["red", "yellow", "blue", "grey", "black", "purple", "pink",
             "brown", "green", "orange"] * 2  # Spectral9
#color_key = color_key[:len(set(preds))+1]

means = gmm.means_

print(np.bincount(preds))
print len(np.bincount(preds))
print(gmm.weights_)

NK [27.0, 9.0, 26.0, 6.0, 32.0, 10.0, 22.0, 29.0, 26.0, 23.0, 15.0, 35.0, 12.0, 1.0, 25.0, 23.0, 27.0, 7.0, 1.0, 4.0, 29.0, 38.0, 22.0, 8.0, 29.0, 41.0, 20.0, 50.0, 12.0, 30.0, 30.0, 36.0, 11.0, 12.0, 8.0, 9.0, 23.0, 36.0, 10.0, 39.0, 17.0, 2.0, 17.0, 17.0, 29.0, 30.0, 7.0, 36.0, 8.0, 18.0]
-inf 435.928006154
8.00270072432e-09 435.928006162
NK [27.0, 9.0, 26.0, 6.0, 32.0, 10.0, 22.0, 29.0, 26.0, 23.0, 15.0, 35.0, 12.0, 1.0, 25.0, 23.0, 27.0, 7.0, 1.0, 4.0, 29.0, 38.0, 22.0, 8.0, 29.0, 41.0, 20.0, 50.0, 12.0, 30.0, 30.0, 36.0, 11.0, 12.0, 8.0, 9.0, 23.0, 36.0, 10.0, 39.0, 17.0, 2.0, 17.0, 17.0, 29.0, 30.0, 7.0, 36.0, 8.0, 18.0]
-inf 435.928006154
8.00270072432e-09 435.928006162
NK [27.0, 9.0, 26.0, 6.0, 32.0, 10.0, 22.0, 29.0, 26.0, 23.0, 15.0, 35.0, 12.0, 1.0, 25.0, 23.0, 27.0, 7.0, 1.0, 4.0, 29.0, 38.0, 22.0, 8.0, 29.0, 41.0, 20.0, 50.0, 12.0, 30.0, 30.0, 36.0, 11.0, 12.0, 8.0, 9.0, 23.0, 36.0, 10.0, 39.0, 17.0, 2.0, 17.0, 17.0, 29.0, 30.0, 7.0, 36.0, 8.0, 18.0]
-inf 435.928006154
8.0

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

from sklearn.decomposition import PCA
pca = PCA(n_components=3)
X_pca = pca.fit(X).transform(X)

In [ ]:
data_thr['pca1'] = X_pca[:, 0]
data_thr['pca2'] = X_pca[:, 1]
data_thr['pca3'] = X_pca[:, 2]

means =  gmm.means_
covs = gmm.covariances_
print 'set(preds)=', set(preds)
print 'gmm.weights_=', gmm.weights_

#data5 = pd.DataFrame({name: data_thr[name] for name in ['0', '20', '40', '70', 'preds']})
data5 = pd.DataFrame({name: data_thr[name] for name in ['pca1', 'pca2', 'pca3', 'preds']})

from sklearn.decomposition import PCA
pca = PCA(n_components=3)
X_pca = pca.fit(X).transform(X)

covs_ = np.array([pca.transform(pca.transform(covs[j].T).T) for j in range(covs.shape[0])])
means_ = np.array([pca.transform(means[j].reshape(1,-1)).T for j in range(means.shape[0])])

p = scatter_matrix(data5, covs=covs_, means=means_, color_key=color_key, covs_indices=['pca1', 'pca2', 'pca3'],
                   plot_width=200, plot_height=200, spread=True, datashader=False)
show(p)

In [ ]:
%matplotlib inline
plt.figure()
plt.plot(pca.components_.T)
plt.show()

In [ ]:
x = 5
y = 0
covs_xy = [covs[j][[x, y]][:, [x, y]] for j in range(len(covs))]
means_xy = [means[j][[x, y]] for j in range(len(covs))]

p = bokeh_plot_cov(data_thr, covs=covs_xy, means=means_xy,
                   x_name='5',
                   y_name='0',
                   pred_name='preds', title=None,  # try preds -> y
                   plot_width=900, plot_height=300)
show(p)